<a href="https://colab.research.google.com/github/JayYongjaeKim/MoLab/blob/main/test_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

aiffel_ds_3_house_prices_path = kagglehub.competition_download('aiffel-ds-3-house-prices')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. 데이터 이해 및 탐색


## 1-1. 데이터 import 및 기본 데이터 확인

In [ ]:
train_path = '/kaggle/input/aiffel-ds-3-house-prices/train.csv'
test_path = '/kaggle/input/aiffel-ds-3-house-prices/test.csv'

import pandas as pd

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

#원본 확인 용 보존
train = train_df.copy()
test = test_df.copy()

In [ ]:
train_df.head(1)
test_df.head(1)

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
#merge를 위한 전처리

train_df['is_train'] = 1
test_df['is_train'] = 0

test_df['SalePrice'] = 0

In [ ]:
#merge를 통해 test데이터 학습

merged_df = pd.concat([train_df, test_df], axis=0)

In [ ]:
#PCA or 상위상관관계만 학습 고려

merged_df.info()

In [ ]:
#75% 월등히 넘어가는 컬럼들 확인 > 이상치 상기

merged_df.describe()

In [ ]:
merged_df.isna().sum().sort_values(ascending=False)

## 요약
> - 주택관련 내부 모든 데이터들에 대해 수치형과 범주형으로 나타냄
> - 변수가 많아 상관관계 및 PCA를 통한 학습률 상승 고민 필요
> - 이상치가 극히 높은 데이터들이 존재해 처리방안 고민 필요
> - null 값에 관련된 데이터 고민 필요
> - 수치형과 범주형을 어떻게 학습시킬지 스케일링 고민 필요

# 2. 데이터 전처리(EDA)

In [ ]:
# 상위 널값 제거 필요

merged_df.isna().sum().sort_values(ascending=False)

In [ ]:
# 널 비중 확인

null_col = merged_df.columns[merged_df.isna().sum() > 0]

null_ratio = {}

for col in null_col:
    ratio = round((merged_df[col].isna().sum() / len(merged_df)) * 100, 2)
    null_ratio[col] = ratio

null_ratio

In [ ]:
# FireplaceQu 기준 컬럼 모두 삭제 (null over 47%)

null_over= [key for key, value in null_ratio.items() if value > 46]

merged_df = merged_df.drop(null_over, axis=1)

merged_df.columns

In [ ]:
# 문자형 null 확인

null_col_test = merged_df.columns[merged_df.isna().sum() > 0]
merged_df[null_col_test].dtypes

In [ ]:
# 비중 확인  *이후 확인시 LotFrontage 상관관계 낮다면 drop 고려

filtered_null_ratio = {key: value for key, value in null_ratio.items() if key in null_col_test}

filtered_null_ratio

In [ ]:
# 결측치 비율이 굉장이 낮음 (신규 방법 적용 이후 재학습)

# null_columns = [key for key, values in filtered_null_ratio.items()]
# merged_df.dropna(null_columns, axis=0) >> test셋 고려 불가 방안(or is_train==1만 삭제)

# numeric_cols = merged_df.select_dtypes(include=['float64', 'int64']).columns
# for col in null_num_columns:
#     merged_df[col].fillna(mergeed_df[col].mean(), inplace=True)

# 신규 방법
from sklearn.impute import SimpleImputer

numeric_cols = merged_df.select_dtypes(include=['float64', 'int64']).columns
object_cols = merged_df.select_dtypes(include=['object']).columns

numeric_imputer = SimpleImputer(strategy='median')
merged_df[numeric_cols] = numeric_imputer.fit_transform(merged_df[numeric_cols])

object_imputer = SimpleImputer(strategy='most_frequent')
merged_df[object_cols] = object_imputer.fit_transform(merged_df[object_cols])

merged_df.isna().sum().sort_values(ascending=False)

In [ ]:
# 이상치 확인  > 그래프로 확인하기 어려워 신규 방안 적용(이후 재학습)

numeric_col_test = merged_df.select_dtypes(include = ['float64', 'int'])

outlier_col = []
outlier_ratio = {}

for col in numeric_col_test:
    Q1 = merged_df[col].quantile(0.25)
    Q3 = merged_df[col].quantile(0.75)
    IQR = Q3 - Q1

    outlier = merged_df[(merged_df[col] < (Q1 -  1.5 * IQR)) | (merged_df[col] > (Q3 + 1.5 * IQR))]

    if not outlier.empty:
        outlier_col.append(col)

    outlier_ratio[col] = len(outlier) / len(merged_df)


print(outlier_col, "\n", outlier_ratio)


In [ ]:
out = pd.DataFrame(list(outlier_ratio.items()), columns=['Column', 'OutlierRatio'])
out.sort_values('OutlierRatio', ascending=False)

In [ ]:
# 문자형 변수 인코딩

merged_df = pd.get_dummies(merged_df, columns=merged_df.select_dtypes(include=['object']).columns)

merged_df.head()

In [ ]:
# 이상치를 제거하는 것이 맞는가?
# 모델평가를 지금 해야하는 것일까?

# 이상치 제거 전 모델 평가

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

train_set = merged_df[merged_df['is_train'] == 1]
test_set = merged_df[merged_df['is_train'] == 0]

x = train_set.drop(columns=['SalePrice', 'is_train'])
y = train_set['SalePrice']


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123)
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)


# 평가지표 코드 루틴화
def r_e(a, b):
    mae = mean_absolute_error(a, b)
    mse = mean_squared_error(a, b)
    rmse = np.sqrt(mse)
    r2 = r2_score(a, b)

    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    print(f"R^2: {r2}")



print("이상치 제거 전")
r_e(y_test, y_pred)



#이상치 제거 후 모델 학습 및 평가
for col in outlier_col:
    Q1 = train_set[col].quantile(0.25)
    Q3 = train_set[col].quantile(0.75)
    IQR = Q3 - Q1
    outlier_condition = (train_set[col] < (Q1 - 1.5 * IQR)) | (train_set[col] > (Q3 + 1.5 * IQR))
    train_set = train_set[~outlier_condition]

merged_df2 = pd.concat([train_set, test_set], axis=0)

x = train_set.drop(columns=['SalePrice', 'is_train'])
y = train_set['SalePrice']


x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123)
model = LinearRegression()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)


print("이상치 제거 후")
r_e(y_test, y_pred)


#제거하기로 결정

In [ ]:
# 이상치 제거

train_set = merged_df[merged_df['is_train']==1]
test_set = merged_df[merged_df['is_train']==0]

for col in outlier_col:
    Q1 = train_set[col].quantile(0.25)
    Q3 = train_set[col].quantile(0.75)
    IQR = Q3 - Q1

    outlier_condition = (train_set[col] < (Q1 - 1.5 * IQR)) | (train_set[col] > (Q3 + 1.5 * IQR))

    train_set = train_set[~outlier_condition]

merged_df = pd.concat([train_set, test_set], axis=0)
merged_df.reset_index(drop=True, inplace=True)
merged_df



In [ ]:
#변수가 많을땐 타겟기준 corr 보기

numeric_df = merged_df[merged_df['is_train'] == 1].select_dtypes(include=['float64', 'int64'])
corr_mat = numeric_df.corr()

target_corr = corr_mat['SalePrice'].apply(abs).sort_values(ascending=False)
target_corr.head(21)


# 품질, 생활공간(지상), 차고, 주택연도, 욕실수, 차고 건축연도, 차고면적, 리모델링연도, 지상층방수, 지하면적 정도까지 상관관계 있다 판별

In [ ]:
# 상관괸계 높은 데이터만 우선 확인 > 스케일링 고려

import matplotlib.pyplot as plt

cols_test = target_corr.head(10).index

for col in cols_test:
    plt.figure(figsize=(3, 2))
    plt.hist(merged_df[col].dropna(), bins=30, edgecolor='k')
    plt.xlabel(col)
    plt.ylabel('fq')
    plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

merge_df2 = merged_df.copy()

# 정규 분포를 따름
standard = ['OverallQual', 'GrLivArea', 'FullBath', 'GarageCars', 'TotRmsAbvGrd']
s_scaler = StandardScaler()
merged_df2[standard] = s_scaler.fit_transform(merged_df2[standard])

# 연속적인 시간의 흐름, 날짜차이 반영
minmax = ['GarageArea', 'YearBuilt', 'GarageYrBlt', 'YearRemodAdd']
m_scaler = MinMaxScaler()
merged_df2[minmax] = m_scaler.fit_transform(merged_df2[minmax])




In [ ]:
# merged_df.head(1)
# merged_df.shape
merged_df[merged_df['is_train']==0].shape

In [ ]:
merged_df[cols_test].dtypes

In [ ]:
# 문자형을 포함한 상관관계

corr_mat2 = merged_df.corr()

target_corr = corr_mat2['SalePrice'].apply(abs).sort_values(ascending=False)
target_corr.head(21)

high_corr_features = target_corr.head(11)

high_corr_features[1:]

# 수치형은 정규화 표준화 해서 상관관계가 낮아지고 , 문자형은 원핫인코딩을 진행해서 상위권에 진입하게 되었는데 이는 학습에 영향을 주지 않는가?

# 3. 모델 선택 및 비교

In [ ]:
# 선형회귀  문자형 포함

top_10_features = high_corr_features.index.drop('SalePrice')

x = merged_df[merged_df['is_train'] == 1][top_10_features]
y = merged_df[merged_df['is_train'] == 1]['SalePrice']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123)

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)

r_e(y_test, y_pred)


In [ ]:
# 릿지 문자형 포함

from sklearn.linear_model import Ridge

rg = Ridge(alpha=1.0)
rg.fit(x_train, y_train)

y_pred = rg.predict(x_test)

r_e(y_test, y_pred)

In [ ]:
# 라쏘 문자형 포함
from sklearn.linear_model import Lasso

ls = Lasso(alpha=0.01)
ls.fit(x_train, y_train)

y_pred = ls.predict(x_test)

r_e(y_test, y_pred)

In [ ]:
# 선형회귀 수치형만

top_10_features2 = target_corr.index.drop('SalePrice')

x2 = merged_df[merged_df['is_train'] == 1][top_10_features2]
y2 = merged_df[merged_df['is_train'] == 1]['SalePrice']

x_train, x_test, y_train, y_test = train_test_split(x2, y2, test_size=0.3, random_state=123)

lr = LinearRegression()
lr.fit(x_train, y_train)

y_pred = lr.predict(x_test)

r_e(y_test, y_pred)


In [ ]:
# 릿지

from sklearn.linear_model import Ridge

rg = Ridge(alpha=1.0)
rg.fit(x_train, y_train)

y_pred = rg.predict(x_test)

r_e(y_test, y_pred)

In [ ]:
# 라쏘
from sklearn.linear_model import Lasso

ls = Lasso(alpha=0.01)
ls.fit(x_train, y_train)

y_pred = ls.predict(x_test)

r_e(y_test, y_pred)

In [ ]:
# 랜덤 포레스트
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=100, random_state=123)
rf.fit(x_train, y_train)

y_pred = rf.predict(x_test)

r_e(y_test, y_pred)

In [ ]:
# XGBoost
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators=100, learning_rate=0.05, max_depth=3, random_state=123)
xgb.fit(x_train, y_train)

y_pred = xgb.predict(x_test)

r_e(y_test, y_pred)

In [ ]:
#스태킹
from sklearn.ensemble import StackingRegressor
import lightgbm as lgb

base_learners = [
    ('rf', RandomForestRegressor(n_estimators=100, random_state=123)),
    ('xgb', XGBRegressor(n_estimators=100, learning_rate=0.05, max_depth=3, random_state=123)),
    ('lgb', lgb.LGBMRegressor(n_estimators=100, learning_rate=0.05, max_depth=3, random_state=123))
]

meta = Ridge(alpha=1.0)

stack= StackingRegressor(estimators=base_learners, final_estimator=meta)

stack.fit(x_train, y_train)

y_pred = stack.predict(x_test)

r_e(y_test, y_pred)

## 3-1 요약
> - 모두 포함(범주형(문자형)과 수치형) : 모두 성과가 좋지 않음 상관관계가 영향이 확실히 나타나는걸 확인하여 표준화 정규화등 스케일링을 진행 후 상관관계가 퍼질 수있지만 낮아진다면 고려대상이다
> - 수치형       :  모두 성과가 좋게 나타남 Rinear_regression은 전체 재실행 하니 갑자기 R2스코어가 극히 나쁘게 나옴

- 이를 반영한 수정작업 진행

# 4. 성능 평가

> - 새로운 피쳐 생성 적용(상관관계 높은 데이터 곱해서 적용)
> - sales 값 로그화
> - 하이퍼파라미터 튜닝

In [ ]:
from sklearn.model_selection import GridSearchCV

ridge = Ridge()

param_grid = {
    'alpha': [0.1, 0.5, 1.0, 5.0, 10.0, 50.0, 100.0]
}

gs= GridSearchCV(estimator=ridge, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

gs.fit(x_train, y_train)

print(f"Best alpha: {gs.best_params_['alpha']}")

y_pred = gs.best_estimator_.predict(x_test)

r_e(y_test, y_pred)


In [ ]:
# warnings.filterwarnings("ignore", category=UserWarning, message=".*DataFrame is highly fragmented.*")

# new_features = [f"{top_10_features2[i]}_{top_10_features2[j]}_prod"
#                 for i in range(len(top_10_features2)-1)
#                 for j in range(i+1, len(top_10_features2))]

# new_feature_data = {}

# for i in range(len(top_10_features2)-1):
#     for j in range(i+1, len(top_10_features2)):
#         new_feature_name = f"{top_10_features2[i]}_{top_10_features2[j]}_prod"
#         new_feature_data[new_feature_name] = merged_df[top_10_features2[i]] * merged_df[top_10_features2[j]]

# new_feature_df = pd.DataFrame(new_feature_data)

# merged_df = pd.concat([merged_df, new_feature_df], axis=1)

# x = merged_df[merged_df['is_train'] == 1][new_features]
# y = merged_df[merged_df['is_train'] == 1]['SalePrice']

# x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=123)

# ridge = Ridge(alpha=10.0)
# ridge.fit(x_train, y_train)

# y_pred = ridge.predict(x_test)

# r_e(y_test, y_pred)

#최악

In [ ]:
x2 = merged_df[merged_df['is_train'] == 1][top_10_features2]
y2 = merged_df[merged_df['is_train'] == 1]['SalePrice']

x_train, x_test, y_train, y_test = train_test_split(x2, y2, test_size=0.3, random_state=123)

ridge = Ridge(alpha=10.0)

ridge.fit(x_train, y_train)

y_pred = ridge.predict(x_test)

r_e(y_test, y_pred)

test_set = merged_df[merged_df['is_train'] == 0]
test_set_cleaned = test_set[x_train.columns]

y_test_pred = ridge.predict(test_set_cleaned)

submission = pd.DataFrame({
    'Id': test_set['Id'],
    'SalePrice': y_test_pred
})
submission['Id'] = submission['Id'].astype('Int32')


submission.to_csv('submission.csv', index=False)
print("Kaggle 생성 완료")

In [ ]:
test_set

In [ ]:
x_train

In [ ]:
train_columns = x_train.columns
test_columns = test_set.columns

missing_in_test = train_columns.difference(test_columns)
missing_in_train = test_columns.difference(train_columns)

print("x_train에만 있는 컬럼:", missing_in_test)
print("test_set에만 있는 컬럼:", missing_in_train)

In [ ]:
submission

In [ ]:
submission.dtypes

In [ ]:
import os
print(os.listdir('/kaggle/working/'))
